In [36]:
from gurobipy import Model, GRB
import pandas as pd

In [37]:
# Load your data
costs_df = pd.read_csv('/Users/mahinbindra/Downloads/costs.csv')  # Ensure you set the correct path to the file

In [38]:
# Initialize the Gurobi model
model = Model("HealthLink_Supplies_Distribution")

In [39]:
# Define the number of sites
num_sites = len(costs_df)

In [40]:
# Decision Variables
# x_i: Binary variable for establishing a warehouse at site i
# y_i: Continuous variable for the units stored at site i, if a warehouse is established
x = model.addVars(num_sites, vtype=GRB.BINARY, name="x")
y = model.addVars(num_sites, vtype=GRB.CONTINUOUS, name="y")


In [41]:
# Objective Function: Minimize total cost (fixed costs and transportation costs)
model.setObjective(sum(costs_df['Fixed'][i] * x[i] + costs_df['Variable'][i] * y[i] for i in range(num_sites)), GRB.MINIMIZE)

In [42]:
# Constraints
# Capacity constraints as a separate constraint
for i in range(num_sites):
    model.addConstr(y[i] <= 375000 * x[i], f"Upper_Capacity_{i+1}")
    model.addConstr(y[i] >= 175000 * x[i], f"Lower_Capacity_{i+1}")

# Mutual exclusion constraints based on the image
for j in [5, 6, 7]:
    model.addConstr(y[0] <= 1 - x[j], f"Mutual_Exclusion_1_{j+1}")
    model.addConstr(y[1] <= 1 - x[j], f"Mutual_Exclusion_2_{j+1}")

for i_prime in [24, 26, 27]:
    model.addConstr(sum(x[i] for i in range(18, 22)) <= 4 * (1 - x[i_prime-1]), f"Mutual_Exclusion_19_22_{i_prime}")

# Constraint 6: If any location from 1-5 is chosen, at least one odd site from 21-27 must be chosen
odd_sites = [i for i in range(20, 27) if i % 2 == 1]
model.addConstr(sum(x[i] for i in range(5)) <= sum(x[i] for i in odd_sites), "odd_sites_selection")

# Constraint 7: Equal number of locations in 1-14 and 15-27
model.addConstr(sum(x[i] for i in range(14)) == sum(x[i] for i in range(14, 27)), "equal_distribution_sites")

# Constraint 8: Equal sum of units in 1-9 and 19-27
model.addConstr(sum(y[i] for i in range(9)) == sum(y[i] for i in range(18, 27)), "equal_units_distribution")

# 5. Total units constraint
model.addConstr(sum(y[i] for i in range(num_sites)) == 2900000, "Total_Units")

<gurobi.Constr *Awaiting Model Update*>

In [43]:
# Optimize the model
model.optimize()


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 67 rows, 54 columns and 215 nonzeros
Model fingerprint: 0x21480d4b
Variable types: 27 continuous, 27 integer (27 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+05]
  Objective range  [2e-01, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+06]
Found heuristic solution: objective 1.888275e+07
Presolve removed 10 rows and 4 columns
Presolve time: 0.00s
Presolved: 57 rows, 50 columns, 187 nonzeros
Variable types: 25 continuous, 25 integer (25 binary)

Root relaxation: objective 1.199367e+07, 33 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.1994e+07    0    2 1.8883e+07 1.1994e+07  36.5%    

In [44]:
# Display the output
if model.status == GRB.OPTIMAL:
    print("\nOptimal Solution Found:")
    for v in model.getVars():
        print(f"{v.VarName}, {v.X}")
else:
    print("\nNo optimal solution found.")


Optimal Solution Found:
x[0], 0.0
x[1], 0.0
x[2], -0.0
x[3], -0.0
x[4], 1.0
x[5], -0.0
x[6], 1.0
x[7], -0.0
x[8], 1.0
x[9], -0.0
x[10], 1.0
x[11], -0.0
x[12], -0.0
x[13], -0.0
x[14], -0.0
x[15], 1.0
x[16], -0.0
x[17], -0.0
x[18], 1.0
x[19], -0.0
x[20], -0.0
x[21], 1.0
x[22], -0.0
x[23], -0.0
x[24], 1.0
x[25], -0.0
x[26], -0.0
y[0], 0.0
y[1], 0.0
y[2], 0.0
y[3], 0.0
y[4], 375000.0
y[5], 0.0
y[6], 325000.0
y[7], 0.0
y[8], 375000.0
y[9], 0.0
y[10], 375000.0
y[11], 0.0
y[12], 0.0
y[13], 0.0
y[14], 0.0
y[15], 375000.0
y[16], 0.0
y[17], 0.0
y[18], 325000.0
y[19], 0.0
y[20], 0.0
y[21], 375000.0
y[22], 0.0
y[23], 0.0
y[24], 375000.0
y[25], 0.0
y[26], 0.0
